https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf

In [39]:
import pandas as pd
import requests
import re, csv

In [6]:
dfAll = pd.read_csv('./data/NCSBE/ncvoter_Wake.zip')
numRecs = dfAll.shape[0]

In [ ]:
fromID = 0
toID = 0+100
address_set = dfAll.iloc[fromID:toID,2:-1].dropna(how='any')

In [10]:
address_set.to_csv('Addresses.csv'.format(fromID),index=True,header=False)

In [4]:
#census curl --form addressFile=@localfile.csv --form  benchmark=9
theURL = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch --output geocoderesult.csv'
command = 'curl --form addressFile=@localfile.csv --form  benchmark=9'

array(['27604', '28412', '27518', ..., '275021058', '90069', '286077742'], dtype=object)

In [28]:
url = 'http://geocoding.geo.census.gov/geocoder/locations/addressbatch'
payload = {'benchmark':'Public_AR_Current','vintage':'Current_Current'}
files = {'addressFile': ('Addresses.csv', open('Addresses.csv', 'r'), 'text/csv')}
r = requests.post(url, files=files, data = payload)

In [32]:
url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch?form'
payload = {'benchmark':'Public_AR_Current','vintage':'Current_Current'}
files = {'addressFile': open('Addresses.csv')}
r = requests.post(url, files=files, data = payload)

In [42]:
r.text.split("\n")[0]#[1:-2]

'"88","2002 BARRINGTON MANOR DR # 105, RALEIGH, NC, 27612","Match","Exact","2002 BARRINGTON MANOR DR, RALEIGH, NC, 27612","-78.670494,35.846638","612814022","R","37","183","052505","1013"'

In [37]:
 # geocode each spreadsheet 
 def censusGeocode(file, output):
     url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch?form'
     payload = {'benchmark':'Public_AR_Current',
                'vintage':'Current_Current',}
     files = {'addressFile': open(file)}
     r = requests.post(url, files=files, data=payload)
     results = str(r.text)
     results = re.sub('"','',results)
     results = results.split('\n')
     with open(output, 'w', newline = '') as geocodeOutput:
         w = csv.writer(geocodeOutput, delimiter=',')
         w.writerows([c.strip() for c in r.split(',')] for r in results)

In [40]:
 censusGeocode('Addresses.csv', 'censusOutput1.csv')